In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns


load preprocess

In [ ]:
# Load and preprocess the data
data_generator = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

data_dir = '/content/drive/MyDrive/mel3rddatset'
batch_size = 32

train_generator = data_generator.flow_from_directory(
    data_dir,
    target_size=(229, 229),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

validation_generator = data_generator.flow_from_directory(
    data_dir,
    target_size=(229, 229),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)


load pretrained model

In [ ]:
# Load pre-trained model (GoogleNet)
base_model = InceptionV3(weights='imagenet', include_top=False)

87910968/87910968 [==============================] - 0s 0us/step


In [ ]:
# Add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)

# Add a fully connected layer
x = Dense(1024, activation='relu')(x)

# Add an output layer with 2 units for binary classification
predictions = Dense(2, activation='softmax')(x)

# Combine the base model and the custom layers
model = Model(inputs=base_model.input, outputs=predictions)
# Fine-tune the model
for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])


Training

In [ ]:
# Train the model

history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=35
)

In [ ]:
# plot acc loss matrics

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Training', 'Validation'], loc='lower right')
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Training', 'Validation'], loc='upper right')
plt.show()


save model

In [ ]:
#Save the model
model.save('/content/drive/MyDrive/Data Mela/gn3rdmeladata_model.h5')


testing

In [ ]:
# Testing model on test data
test_generator = data_generator.flow_from_directory(
    data_dir,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False,
    subset='validation'
)

# evaluate model on test set
test_loss, test_acc = model.evaluate(test_generator)
print('Test loss:', test_loss)
print('Test accuracy:', test_acc)

# Predict using the model
y_pred = model.predict(test_generator)

# Convert predictions to class labels
y_pred_classes = tf.argmax(y_pred, axis=1)

# Get true labels
y_true = test_generator.classes


performance evaluatuion

In [ ]:

report = classification_report(y_true, y_pred_classes, target_names=['benign', 'malignant'])
print(report)


In [ ]:

# Calculate confusion matrix
cm = confusion_matrix(y_true, y_pred_classes)

# Plot the confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=['Benign (0)', 'Malignant (1)'],
            yticklabels=['Benign (0)', 'Malignant (1)'])
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()


In [ ]:
# plot AUC

import numpy as np
from sklearn.metrics import roc_curve, roc_auc_score

fpr, tpr, thresholds = roc_curve(y_true, y_pred_classes)
auc = roc_auc_score(y_true, y_pred_classes)

plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, label='ROC curve (AUC = %0.2f)' % auc)
plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc='lower left')
plt.show()
